In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load the training and test datasets
training_data = pd.read_csv('/content/drive/MyDrive/training.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test_X.csv')

# Separating features and target variable in training data
X_train = training_data.drop(columns=['lettr'])
y_train = training_data['lettr']

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_data)

# Convert scaled features back to DataFrame for better handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=test_data.columns)

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled_df, y_train, test_size=0.2, random_state=42)

# Simulate outliers for validation purposes
outliers_fraction = 0.1
n_outliers = int(outliers_fraction * len(X_val_split))
normal_data = X_val_split[:-n_outliers]
outlier_data = X_val_split[-n_outliers:]
y_val_split = np.hstack([np.zeros(len(normal_data)), np.ones(len(outlier_data))])

# Combine normal data and synthetic outliers
X_val_split = np.vstack([normal_data, outlier_data])


# Method : K-Nearest Neighbors (KNN)
num_neighbors = 1
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(X_train_split)
distances, _ = knn.kneighbors(X_val_split)
mean_distances = distances.mean(axis=1)
knn_auc = roc_auc_score(y_val_split, -mean_distances)

# Train on full data and predict test set
knn.fit(X_train_scaled_df)
distances, _ = knn.kneighbors(X_test_scaled_df)
mean_distances = distances.mean(axis=1)
knn_probabilities = (mean_distances - mean_distances.min()) / (mean_distances.max() - mean_distances.min())
knn_submission = pd.DataFrame({'id': range(len(knn_probabilities)), 'Outliers': knn_probabilities})
knn_submission_path = '/content/drive/MyDrive/submission_knn.csv'
knn_submission.to_csv(knn_submission_path, index=False)



print(f"KNN AUC: {knn_auc}")
print(f"KNN submission saved to {knn_submission_path}")



KNN AUC: 0.5069995590828924
KNN submission saved to /content/drive/MyDrive/submission_knn.csv


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
